In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import time

In [2]:
url = "https://data.ca.gov/dataset/590188d5-8545-4c93-a9a0-e230f0db7290/resource/926fd08f-cc91-4828-af38-bd45de97f8c3/download/statewide_cases.csv"

In [3]:
df = pd.read_csv(url)

### Clean Up

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7325 entries, 0 to 7324
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               7325 non-null   object 
 1   totalcountconfirmed  7322 non-null   float64
 2   totalcountdeaths     7323 non-null   float64
 3   newcountconfirmed    7325 non-null   int64  
 4   newcountdeaths       7325 non-null   int64  
 5   date                 7325 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 343.5+ KB


In [5]:
df.head(10)

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,151.0,6.0,151,6,2020-03-18
1,Santa Clara,183.0,8.0,32,2,2020-03-19
2,Santa Clara,246.0,8.0,63,0,2020-03-20
3,Santa Clara,269.0,10.0,23,2,2020-03-21
4,Santa Clara,284.0,13.0,15,3,2020-03-22
5,Santa Clara,336.0,13.0,52,0,2020-03-23
6,Santa Clara,389.0,17.0,53,4,2020-03-24
7,Santa Clara,452.0,20.0,63,3,2020-03-25
8,Santa Clara,487.0,20.0,35,0,2020-03-26
9,Santa Clara,557.0,25.0,70,5,2020-03-27


In [6]:
df.fillna(0, inplace = True)
df.drop(df[(df['county'] == 'Out Of Country')].index, inplace = True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7227 entries, 0 to 7324
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               7227 non-null   object 
 1   totalcountconfirmed  7227 non-null   float64
 2   totalcountdeaths     7227 non-null   float64
 3   newcountconfirmed    7227 non-null   int64  
 4   newcountdeaths       7227 non-null   int64  
 5   date                 7227 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 395.2+ KB


In [8]:
county_df = df[df.groupby('county').date.transform('max') == df['date']]
county_df.reset_index(drop = True)
# county_df.drop(columns='index')
county_df.head(10)

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
122,Santa Clara,7255.0,177.0,175,0,2020-07-18
245,San Mateo,4488.0,114.0,81,0,2020-07-18
368,Santa Barbara,5110.0,32.0,109,0,2020-07-18
491,Tuolumne,75.0,0.0,2,0,2020-07-18
613,Sierra,1.0,0.0,0,0,2020-07-18
736,Placer,1233.0,11.0,26,0,2020-07-18
859,San Luis Obispo,1234.0,6.0,44,0,2020-07-18
982,Solano,2503.0,31.0,51,0,2020-07-18
1105,Monterey,3020.0,18.0,98,0,2020-07-18
1228,Yuba,237.0,3.0,6,0,2020-07-18


In [9]:
county_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 122 to 7324
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               59 non-null     object 
 1   totalcountconfirmed  59 non-null     float64
 2   totalcountdeaths     59 non-null     float64
 3   newcountconfirmed    59 non-null     int64  
 4   newcountdeaths       59 non-null     int64  
 5   date                 59 non-null     object 
dtypes: float64(2), int64(2), object(2)
memory usage: 3.2+ KB


### Load to SQL

In [10]:
rds_connection_string = "postgres:postgres@localhost:5432/covid-vaccine-plz"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
engine.table_names()

['citywide_table', 'statewide_table']

In [12]:
county_df.to_sql(name='statewide_table',con=engine,if_exists='append',index=False)

In [13]:
pd.read_sql_query('select* from statewide_table',con=engine).head()

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,7080,177,90,1,2020-07-17
1,San Mateo,4407,114,71,0,2020-07-17
2,Santa Barbara,5001,32,129,0,2020-07-17
3,Tuolumne,73,0,2,0,2020-07-17
4,Sierra,1,0,0,0,2020-07-17


In [14]:
# EXPORT TO .CSV
df.to_csv("./CSV/statewide.csv", index=False)